# Rock Paper Scissors Animated Shake-up Plot

**Note**: *This is a lightly edited copy of [Santa 2020 Animated Shake-up Plot](https://www.kaggle.com/jtrotman/santa-2020-animated-shake-up-plot)*

The shake-up was a live, dynamic process in this competition! I entered late after entering the Santa 2020 competition (because I could re-use code and scripts) and so I added RPS to my cron job to poll the leaderboard: making regular snapshots of it, every two hours.

Thanks to the [command line API](https://github.com/Kaggle/kaggle-api) it only takes a minute to upload them as a dataset :)

Then, with a a quick copy/paste job (code from
https://www.kaggle.com/jtrotman/meta-kaggle-scatter-plot-competition-shake-up
which makes a plot for *every* competition!) I've turned them into animations.

(I cannot, in the time available, make the `matplotlib.animation` code work without glitches or flaws of some kind so I use *ImageMagick* `convert` command line. If anyone can fix this to use that API with play/pause/advance buttons etc please do fork and make it public & I'll upvote you.)

In fact, with some more work, the leaderboard could be reconstructed at any point in time from the [Meta Kaggle](https://www.kaggle.com/kaggle/meta-kaggle) data, grouping by team ID and taking the max submission score, after any given episode. If anyone wants to do that then hopefully these leaderboard snapshots can help verify it works and/or spot any intriguing discrepancies.

In [1]:
import json, os, sys, time
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display, Image
from leaderboard_to_csv import *

In [2]:
plt.rc('figure', figsize=(10, 10))
plt.rc('font', size=12)

In [3]:
base_dir = '../input/rock-paper-scissors-leaderboards'
files = sorted(os.listdir(base_dir))
len(files)

In [4]:
with open(f'{base_dir}/{files[-1]}') as f:
    final = json_to_dataframe(json.load(f)).set_index('teamId').add_prefix('final ')

In [5]:
medals = { 'gold':3, 'silver':2, 'bronze':1, None:0 }
medal_colors = np.asarray(['deepskyblue', 'chocolate', 'silver', 'gold'])
rank_to_color = lambda r: f'#00{int((1-r)*255):02x}{int(r*255):02x}'

In [6]:
names = []
for i, json_file in enumerate(files):
    with open(f'{base_dir}/{json_file}') as f:
        df = json_to_dataframe(json.load(f)).set_index('teamId')
        df = df.join(final)
        # using final rankings to color points
        # either color scheme results in bands over x or y coordinates
        # using one consistent color per team makes it easier to see movements
        ranks = df['final rank'].rank(pct=True, ascending=False)
        medal_ser = df['final medal'].map(medals)
        color = ranks.apply(rank_to_color)
        color = np.where(medal_ser, medal_colors[medal_ser], color)
        size = np.log(df['entries']) * 5
        df.plot.scatter('final rank', 'rank', c=color, s=size)
        s = json_file.replace('_', ' ').replace('.json', '')
        date = pd.to_datetime(s)
        plt.title('RPS Shake-up - ' + str(date))
        plt.tight_layout()
        png = f'{i}.png'
        names.append(png)
        plt.savefig(png, bbox_inches='tight')
        plt.close()

In [7]:
# https://github.com/ipython/ipython/issues/10045
def show_gif(fname):
    import base64
    with open(fname, 'rb') as fd:
        b64 = base64.b64encode(fd.read()).decode('ascii')
    return HTML(f'<img src="data:image/gif;base64,{b64}" />')

In [8]:
name_list = ' '.join(names)
repeat_final = ' '.join([names[-1]] * 15)
output = f'rps-shakeup.gif'

In [9]:
!convert -delay 20 -loop 0 -dispose previous {name_list} {repeat_final} {output}

# Animated Shake-up

In [10]:
show_gif(output)

# Top 200 teams

Zoom in on the medal zone

In [11]:
names = []
TOP_N = 200
PAD = 10
for i, json_file in enumerate(files):
    with open(f'{base_dir}/{json_file}') as f:
        df = json_to_dataframe(json.load(f)).set_index('teamId')
        df = df.join(final)
        df = df[df['final rank'] <= TOP_N]
        ranks = df['final rank'].rank(pct=True, ascending=False)
        medal_ser = df['final medal'].map(medals)
        color = ranks.apply(rank_to_color)
        color = np.where(medal_ser, medal_colors[medal_ser], color)
        size = np.log(df['entries']) * 7
        df.plot.scatter('final rank', 'rank', c=color, s=size)
        s = json_file.replace('_', ' ').replace('.json', '')
        date = pd.to_datetime(s)
        plt.title(f'RPS Shake-up - Top {TOP_N} - ' + str(date))
        plt.xlim(-PAD, TOP_N+PAD)
        plt.ylim(-PAD, TOP_N+PAD)
        plt.tight_layout()
        png = f'top{TOP_N}_{i}.png'
        names.append(png)
        plt.savefig(png, bbox_inches='tight')
        plt.close()

In [12]:
name_list = ' '.join(names)
repeat_final = ' '.join([names[-1]] * 15)
output = f'rps-shakeup-top{TOP_N}.gif'

In [13]:
!convert -delay 20 -loop 0 -dispose previous {name_list} {repeat_final} {output}

# Animated Top 200 Shake-up

In [14]:
show_gif(output)

Fork & remove this line if you want all the individual frames :)

In [15]:
!rm *.png 